In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


## Built-in Tools

### Duck-Duck Go Search

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('IPL News')

print(results)

RR vs MI IPL Live (Rajasthan Royals cricket team vs Mumbai Indians cricket team): Follow live commentary for the IPL 2025 match between Rajasthan Royals and Mumbai Indians in Jaipur. Check RR vs MI full scorecard, ball by ball commentary, match highlights, top performers, records and more here. Cricket News: Shreyas Iyer, leading Punjab Kings, is showcasing exceptional form in the IPL, potentially marking his best season yet. His impressive 72-run innings IPL News IPL 2025: Rajasthan Royals' Sandeep Sharma ruled out of remainder of tournament, pacer's replacement to be named soon The Royals management is actively working on finalising Sandeep Sharma's replacement. Published : May 01, 2025 19:09 IST , CHENNAI - 1 MIN READ . Team Sportstar ... Get the latest news, updates, predictions and scores for IPL 2025, the cash-rich T20 cricket league in India. Find out about the mega-auction, the teams, the players, the fixtures, the standings and the fantasy tips for IPL 2025. Punjab Kings (PBKS

In [4]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


### Shell Tool

In [7]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('whoami')

print(results)

Executing command:
 whoami
root



/usr/local/lib/python3.11/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom Tools

In [8]:
from langchain_core.tools import tool

In [9]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [10]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [11]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [12]:
result = multiply.invoke({"a":3, "b":5})

In [13]:
print(result)

15


In [14]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [15]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## StructuredTool

In [1]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [2]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [3]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [4]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [5]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## BaseTool Class

In [6]:
from langchain.tools import BaseTool
from typing import Type

In [7]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [8]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [9]:
multiply_tool = MultiplyTool()

In [10]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit

In [11]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [12]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [13]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
